In [5]:
### Safe to rerun ###

import config
import gspread
import json
import openai
import os

import numpy as np
import pandas as pd

# Authorize gspread
gc, authorized_user = gspread.oauth_from_dict(config.gspread_secret_key_personal, config.gspread_auth_key_personal)

# Read Input from directory

directory = r"C:\Users\Hooman Deghani\Python\Data Analysis\Outreach - Skyscraper\Output\Current"

## List all CSV files in the directory

urls = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.xlsx')]

if len(urls) > 1:
    df = pd.concat([pd.read_excel(url) for url in urls], ignore_index=True)

else: 
    df = pd.read_excel(urls[0])

# Create a variable for the title of the project
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):
        # Capture the cryptic name
        artifact_name = filename
        
        # Extract the essence
        title = artifact_name.split('.')[0]

# Load records.json into a python dictionary titled records
with open(r"C:\Users\Hooman Deghani\Python\Data Analysis\Outreach - Skyscraper\Records.json", "r") as records_json:
    records = json.load(records_json)

# Check if an entry in records exists with title
if title not in records:
    records[title] = {
        'New': 'True',
        'Last': ''
    }

records

{'fthb': {'New': 'False',
  'Last': 'https://www.wkbw.com/financial-fitness/eager-buyers-are-skipping-home-inspections-is-it-too-risky'},
 'wfh': {'New': 'False',
  'Last': 'https://www.ipma.world/how-to-work-remotely/'}}

In [6]:
### Safe to rerun ###
# Clean up the data #

# Make df json compliant
df = df.applymap(str)
df.replace("nan", "", inplace=True)
df = df.applymap(str.strip)

# Remove rows where email is empty
Recipient_null = df.loc[:, 'Recipient'] == ""
df = df.loc[~Recipient_null]

# Reset index
df = df.reset_index(drop=True)



In [7]:
### Safe to rerun ###
# Prepare the batch #

# TODO: drop Anchor, DR
df = df.drop(columns=['Anchor', 'Domain rating'])

# Create the dic variable
dic = records.get(title)

# if dataframe is new, batch = df.loc[0:30,:]
if dic.get("New") == "True":
    batch = df.loc[0:30,:]

# if dataframe is old, batch = the next 30 after "Last"
if dic.get("New") == "False":

    # Get the index label of the last email sent
    filt = df.loc[:, 'Referring page URL'] == records.get(title).get("Last")
    last = df.loc[filt].index[0]

    # batch = 30 after last
    batch = df.loc[(last + 1): (last + 31)]

batch

,Referring page URL,Referring Topic,Target URL,Root URL,First Name,Last Name,Recipient,Email Sent,Status,Replied,Converted
93,https://digitalresilience.northeastern.edu/wfa...,,https://blog.hubspot.com/marketing/productivit...,https://northeastern.edu,Koen,Pauwels,kpauwels@northeastern.edu,,,,
94,https://ombuds.uconn.edu/resources-for-working...,,https://blog.hubspot.com/marketing/productivit...,https://uconn.edu,Morgan,Peniston,morgan.peniston@uconn.edu,,,,
95,https://www.thebulwark.com/how-to-work-from-ho...,,https://blog.hubspot.com/marketing/productivit...,https://thebulwark.com,Tim,Miller,tim@thebulwark.com,,,,
96,https://georgetownchamber.org/coronavirus/,,https://blog.hubspot.com/marketing/productivit...,https://georgetownchamber.org,Cheryl,Brinkmeyer,cheryl@georgetownchamber.org,,,,
97,https://www.mediaupdate.co.za/media/148202/sev...,,https://blog.hubspot.com/marketing/productivit...,https://mediaupdate.co.za,Sir/Madam,,editorial@mediaupdate.co.za,,,,
98,https://www.danby.com/en-uk/blog/working-from-...,,https://blog.hubspot.com/marketing/productivit...,https://danby.com,Victoria,Freeman,vfreeman@danby.com,,,,
99,https://sarahscoop.com/how-to-run-your-busines...,,https://blog.hubspot.com/marketing/productivit...,https://sarahscoop.com,Sarah,Ruhlman,sarah@sarahscoop.com,,,,
100,https://www.lux-review.com/what-to-consider-wh...,,https://blog.hubspot.com/marketing/productivit...,https://lux-review.com,Edward,Faulkner,efaulkner@lux-review.com,,,,
101,https://www.rfchamber.com/coronavirus-business...,,https://blog.hubspot.com/marketing/productivit...,https://rfchamber.com,Meghann,Witthoft,meghann@rfchamber.com,,,,
102,https://www.menstylefashion.com/tips-for-creat...,,https://blog.hubspot.com/marketing/productivit...,https://menstylefashion.com,Sir/Madam,,gracie.opulanza@menstylefashion.com,,,,


In [8]:
### Safe to Rerun ###
# Fill Target Topic & Target Hyperlink

# Create the "Target Topic" column
batch.loc[:, ['Target Topic', 'Target Hyperlink']] = ''


# Get the list of target URLs
print(batch.loc[:, 'Target URL'].value_counts())


# Create filters for each URL
url_1 = "https://blog.hubspot.com/marketing/productivity-tips-working-from-home"
filt_1 = (batch.loc[:, 'Target URL'] == url_1)

url_2 = "https://blog.hubspot.com/marketing/productivity-tips-working-from-home?__hstc=219294534.b2b47713ef61d04c6d6fede4eb0fd448.1582064984829.1583868321116.1583937752741.59&__hssc=219294534.8.1583937752741&__hsfp=1704468437"
filt_2 = (batch.loc[:, 'Target URL'] == url_2)

# Fill out Target Hyperlink
batch.loc[filt_1, 'Target Hyperlink'] = f"""<a href="{url_1}">the working from home guide by Hubspot</a>"""

if filt_2.any():
    batch.loc[filt_2, 'Target Hyperlink'] = f"""<a href="{url_2}">the working from home guide by Hubspot</a>"""

# Fill out Target Topic: prereferences; e.g. the guide by nerdwallet
batch.loc[filt_1, 'Target Topic'] = "the guide by Hubspot"

if filt_2.any():
    batch.loc[filt_2, 'Target Topic'] = "the guide by Hubspot"

https://blog.hubspot.com/marketing/productivity-tips-working-from-home                                                                                                                                                      30
https://blog.hubspot.com/marketing/productivity-tips-working-from-home?__hstc=219294534.b2b47713ef61d04c6d6fede4eb0fd448.1582064984829.1583868321116.1583937752741.59&__hssc=219294534.8.1583937752741&__hsfp=1704468437     1
Name: Target URL, dtype: int64


C:\Users\Hooman Deghani\AppData\Local\Temp\ipykernel_10516\1485668661.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch.loc[:, ['Target Topic', 'Target Hyperlink']] = ''


In [9]:
### Safe to rerun ###
# Playground

batch.head()

,Referring page URL,Referring Topic,Target URL,Root URL,First Name,Last Name,Recipient,Email Sent,Status,Replied,Converted,Target Topic,Target Hyperlink
93,https://digitalresilience.northeastern.edu/wfa...,,https://blog.hubspot.com/marketing/productivit...,https://northeastern.edu,Koen,Pauwels,kpauwels@northeastern.edu,,,,,the guide by Hubspot,"<a href=""https://blog.hubspot.com/marketing/pr..."
94,https://ombuds.uconn.edu/resources-for-working...,,https://blog.hubspot.com/marketing/productivit...,https://uconn.edu,Morgan,Peniston,morgan.peniston@uconn.edu,,,,,the guide by Hubspot,"<a href=""https://blog.hubspot.com/marketing/pr..."
95,https://www.thebulwark.com/how-to-work-from-ho...,,https://blog.hubspot.com/marketing/productivit...,https://thebulwark.com,Tim,Miller,tim@thebulwark.com,,,,,the guide by Hubspot,"<a href=""https://blog.hubspot.com/marketing/pr..."
96,https://georgetownchamber.org/coronavirus/,,https://blog.hubspot.com/marketing/productivit...,https://georgetownchamber.org,Cheryl,Brinkmeyer,cheryl@georgetownchamber.org,,,,,the guide by Hubspot,"<a href=""https://blog.hubspot.com/marketing/pr..."
97,https://www.mediaupdate.co.za/media/148202/sev...,,https://blog.hubspot.com/marketing/productivit...,https://mediaupdate.co.za,Sir/Madam,,editorial@mediaupdate.co.za,,,,,the guide by Hubspot,"<a href=""https://blog.hubspot.com/marketing/pr..."


In [8]:
# # Fill referring topic by gpt-3.5
# # Fill referring Hyperlink

#df.loc[:, "Referring hyperlink"] = ''



    

In [10]:
# Fill "SQ1 URL column"
## first possible url: https://www.squareone.ca/resource-centres/home-buying-selling-moving/buying-a-home-for-the-first-time
## second possible url: https://www.squareone.ca/resource-centres/home-personal-safety/work-from-home-guide
batch.loc[:, 'SQ1 URL'] = 'https://www.squareone.ca/resource-centres/home-personal-safety/work-from-home-guide'

C:\Users\Hooman Deghani\AppData\Local\Temp\ipykernel_10516\1540710788.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch.loc[:, 'SQ1 URL'] = 'https://www.squareone.ca/resource-centres/home-personal-safety/work-from-home-guide'


In [11]:
# # Push to Gspreadsheet
# spreadsheet_title = "Outreach"
# worksheet_title = f"Last:{batch.loc[batch.tail(1).index[0], 'First Name']}{batch.loc[batch.tail(1).index[0], 'Last Name']}"

# spreadsheet = gc.open(spreadsheet_title)
# worksheet = spreadsheet.add_worksheet(title=worksheet_title, rows=batch.shape[0], cols=batch.shape[1])
# worksheet.update([batch.columns.tolist()] + batch.values.tolist())

In [12]:
# Save output to input
file_name = f"{batch.loc[batch.tail(1).index[0], 'Recipient']}.csv"
path = r"C:\Users\Hooman Deghani\Python\Data Analysis\Outreach - Skyscraper\Input\Current\F"

batch.to_csv(path+file_name)

In [17]:
# Update records.json #

# Update records with the new variables
records[title]["New"] = "False"
records[title]["Last"] = batch.loc[batch.tail(1).index[0], 'Referring page URL']

# Push records to records.json
with open(r"C:\Users\Hooman Deghani\Python\Data Analysis\Outreach - Skyscraper\Records.json", "w") as records_json:
    json.dump(records, records_json)